# Superpixels Explorer

Interactively compute and visualize superpixel overlays for `original.png` and `edited.png`.

- Place the images in the same folder as this notebook (or anywhere under it).
- Use the sliders/toggles to tune SLIC parameters and visualization.
- View side-by-side results for both images.

Notes:
- This uses `skimage.segmentation.slic`, `skimage.segmentation.mark_boundaries`, and `skimage.color.label2rgb`.
- If packages are missing, the next cell attempts to install them automatically.

In [7]:
# Imports, setup, and image loading
# If you are missing dependencies, uncomment the pip line below.
# %pip install scikit-image ipywidgets pillow matplotlib

from pathlib import Path
import numpy as np
from skimage.segmentation import slic, mark_boundaries
from skimage import img_as_float
from skimage.io import imread
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, FloatSlider, FloatLogSlider, Checkbox, Dropdown, ToggleButtons, interactive_output, HBox, VBox
from IPython.display import display

IMG_DIR = Path('.')
ORIG_PATH = IMG_DIR / 'original.png'
EDIT_PATH = IMG_DIR / 'edited.png'

def safe_read(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"Image not found: {path}")
    img = imread(path)
    if img.ndim == 2:  # grayscale to RGB
        img = np.stack([img]*3, axis=-1)
    return img_as_float(img)

try:
    original_img = safe_read(ORIG_PATH)
    edited_img = safe_read(EDIT_PATH)
    print('Loaded images:')
    print(' original:', original_img.shape, original_img.dtype)
    print(' edited  :', edited_img.shape, edited_img.dtype)
except FileNotFoundError as e:
    print(e)
    print('')
    print('Please ensure both `original.png` and `edited.png` are present in the notebook directory.')

Loaded images:
 original: (1024, 1024, 4) float64
 edited  : (1024, 1024, 4) float64


In [10]:
# Superpixel computation and interactive UI with cropping
import warnings
warnings.filterwarnings('ignore')

from skimage.color import label2rgb
from skimage.segmentation import find_boundaries

# Controls
n_segments_slider = IntSlider(value=400, min=50, max=5000, step=10, description='n_segments', continuous_update=False)
compactness_slider = FloatLogSlider(value=10.0, base=10, min=-2, max=2, step=0.05, description='compactness', continuous_update=False)
sigma_slider = FloatSlider(value=0.0, min=0.0, max=5.0, step=0.1, description='sigma', continuous_update=False)
enforce_checkbox = Checkbox(value=True, description='enforce_connectivity')
start_label_selector = ToggleButtons(options=[0,1], value=1, description='start_label')

boundary_mode_dropdown = Dropdown(options=['thick','inner','outer','subpixel'], value='thick', description='boundary_mode')
boundary_color_dropdown = Dropdown(options=['yellow','red','blue','green','white','black'], value='yellow', description='boundary_color')
fill_checkbox = Checkbox(value=False, description='fill with avg')
fill_alpha_slider = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='fill_alpha')
crop_border_slider = IntSlider(value=20, min=0, max=200, step=1, description='crop_border', continuous_update=False)

controls = VBox([
    HBox([n_segments_slider, compactness_slider, sigma_slider]),
    HBox([enforce_checkbox, start_label_selector, crop_border_slider]),
    HBox([boundary_mode_dropdown, boundary_color_dropdown]),
    HBox([fill_checkbox, fill_alpha_slider])
])

BOUNDARY_COLORS = {
    'yellow': (1.0, 1.0, 0.0),
    'red': (1.0, 0.0, 0.0),
    'blue': (0.0, 0.4, 1.0),
    'green': (0.0, 1.0, 0.0),
    'white': (1.0, 1.0, 1.0),
    'black': (0.0, 0.0, 0.0),
}

IMAGE_FILES = {
    'original': ORIG_PATH,
    'edited': EDIT_PATH,
}

IMAGE_CACHE = {}
for k, p in IMAGE_FILES.items():
    if p.exists():
        IMAGE_CACHE[k] = safe_read(p)

if len(IMAGE_CACHE) < 2:
    print('Warning: Not all images found. Found keys:', list(IMAGE_CACHE.keys()))

def _ensure_rgba(img):
    # Converts grayscale or RGB to RGBA (alpha=1.0) so we have a consistent alpha channel.
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)
    if img.shape[-1] == 3:
        alpha = np.ones((*img.shape[:2], 1), dtype=img.dtype)
        return np.concatenate([img, alpha], axis=-1)
    return img  # already RGBA

def _pad_to_same_canvas(imgs):
    # Pad all images (RGBA) to the max height/width among them with transparent zeros
    hs = [im.shape[0] for im in imgs]
    ws = [im.shape[1] for im in imgs]
    H = max(hs)
    W = max(ws)
    padded = []
    for im in imgs:
        if im.shape[0] == H and im.shape[1] == W:
            padded.append(im)
            continue
        canvas = np.zeros((H, W, 4), dtype=im.dtype)
        canvas[:im.shape[0], :im.shape[1]] = im
        padded.append(canvas)
    return padded

def _compute_crop_bbox(union_mask, border):
    ys, xs = np.where(union_mask)
    if len(ys) == 0:
        return None  # no content
    y0, y1 = ys.min(), ys.max()
    x0, x1 = xs.min(), xs.max()
    y0 = max(0, y0 - border)
    x0 = max(0, x0 - border)
    y1 = min(union_mask.shape[0]-1, y1 + border)
    x1 = min(union_mask.shape[1]-1, x1 + border)
    return y0, y1, x0, x1

def _crop_all(imgs, bbox):
    if bbox is None:
        return imgs, None
    y0, y1, x0, x1 = bbox
    cropped = [im[y0:y1+1, x0:x1+1] for im in imgs]
    return cropped, (y0, y1, x0, x1)

# Core function
def compute_and_show(n_segments, compactness, sigma, enforce_connectivity, start_label, boundary_mode, boundary_color, fill, fill_alpha, crop_border):
    # Prepare RGBA versions and pad to same canvas
    orig_imgs = []
    for key in ['original','edited']:
        if key in IMAGE_CACHE:
            orig_imgs.append(_ensure_rgba(IMAGE_CACHE[key]))
    if not orig_imgs:
        fig, ax = plt.subplots(1, 1, figsize=(6,4))
        ax.axis('off')
        ax.set_title('No images found')
        plt.show()
        return
    padded = _pad_to_same_canvas(orig_imgs)
    # Union alpha mask (alpha>0 treated as content)
    masks = [(im[...,3] > 0).astype(bool) for im in padded]
    union_mask = np.zeros_like(masks[0])
    for m in masks:
        union_mask |= m
    bbox = _compute_crop_bbox(union_mask, int(crop_border))
    cropped_imgs, used_bbox = _crop_all(padded, bbox)
    overlay_imgs = []
    boundary_imgs = []
    titles = ['original','edited'][:len(cropped_imgs)]
    boundary_rgb_color = BOUNDARY_COLORS.get(boundary_color, (1.0,1.0,0.0))
    for img in cropped_imgs:
        # Separate RGB and alpha
        rgb_for_slic = img[..., :3]
        alpha_channel = img[..., 3:4]
        convert_to_lab = (rgb_for_slic.ndim == 3 and rgb_for_slic.shape[-1] == 3)
        segments = slic(
            rgb_for_slic,
            n_segments=int(n_segments),
            compactness=float(compactness),
            sigma=float(sigma),
            start_label=int(start_label),
            enforce_connectivity=bool(enforce_connectivity),
            convert2lab=convert_to_lab,
            channel_axis=-1,
        )
        base = rgb_for_slic.copy()
        if fill:
            avg = label2rgb(segments, image=rgb_for_slic, kind='avg')
            base = (1.0 - float(fill_alpha)) * rgb_for_slic + float(fill_alpha) * avg
        with_boundaries_rgb = mark_boundaries(
            base, segments, mode=boundary_mode, color=boundary_rgb_color
)
        with_boundaries = np.concatenate([with_boundaries_rgb, alpha_channel], axis=-1)
        boundaries_mask = find_boundaries(segments, mode='outer')
        boundaries_only_rgb = np.zeros_like(rgb_for_slic)
        for c in range(3):
            boundaries_only_rgb[..., c] = boundaries_mask * boundary_rgb_color[c]
        boundaries_alpha = boundaries_mask.astype(float)
        boundaries_only = np.concatenate([boundaries_only_rgb, boundaries_alpha[..., None]], axis=-1)
        overlay_imgs.append(with_boundaries)
        boundary_imgs.append(boundaries_only)
    # Figure
    n = len(overlay_imgs)
    fig, axes = plt.subplots(2, n, figsize=(7*n, 10), squeeze=False)
    crop_info = '' if used_bbox is None else f'crop={used_bbox[2]}:{used_bbox[3]} x {used_bbox[0]}:{used_bbox[1]}'
    for i in range(n):
        ax1 = axes[0, i]
        ax1.imshow(overlay_imgs[i])
        ax1.set_title(f"{titles[i]} overlay (n={n_segments})\n{crop_info}")
        ax1.axis('off')
        ax2 = axes[1, i]
        ax2.imshow(boundary_imgs[i])
        ax2.set_title(f"{titles[i]} boundaries")
        ax2.axis('off')
    plt.tight_layout()
    plt.show()

ui = interactive_output(
    compute_and_show,
    {
        'n_segments': n_segments_slider,
        'compactness': compactness_slider,
        'sigma': sigma_slider,
        'enforce_connectivity': enforce_checkbox,
        'start_label': start_label_selector,
        'boundary_mode': boundary_mode_dropdown,
        'boundary_color': boundary_color_dropdown,
        'fill': fill_checkbox,
        'fill_alpha': fill_alpha_slider,
        'crop_border': crop_border_slider,
    }
)

print('Adjust controls below; output updates automatically (cropping + boundaries-only supported):')
display(controls, ui)

Adjust controls below; output updates automatically (cropping + boundaries-only supported):


Output()